# Redes Recurrentes

Vamos a usar el dataset de IMDB de 50k reviews disponible en [kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gzip
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
import torch

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset,DataLoader


from collections import Counter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange


In [ ]:
nltk.download("stopwords") # Descargamos las stop words

In [ ]:
#Seteamos la semilla
seed =14

In [ ]:
#imdbpd.read_csv('/content/drive/MyDrive/Aprendizaje Profundo/Edición 2023/Notebooks 2023/imdb_dataset.csv')
# Leemos el dataset
# = 'imdb_dataset.csv'
df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Profundo/Edición 2023/Notebooks 2023/imdb_dataset.csv")
df.head()

In [ ]:
#df = df.iloc[:500]

In [ ]:
X = df['review'].values
y = df['sentiment'].values

# Dividir el conjunto de datos en conjuntos de entrenamiento, validación y test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)

print(f'Cantidad de datos en train: {X_train.shape[0]}')
print(f'Cantidad de datos en validación: {X_val.shape[0]}')
print(f'Cantidad de datos en test: {X_test.shape[0]}')

## Pequeña inspección de los datos

In [ ]:
target_dist = pd.Series(y_train).value_counts()
sns.barplot(x=np.array(['negative','positive']),y=target_dist.values)
plt.show()

## Procesamiento de texto


El preprocesamiento de texto y la tokenización son un primer paso fundamental. Primero, limpiamos los datos del texto eliminando puntuación, espacios adicionales y números.

Luego transformamos oraciones en palabras individuales, eliminamos palabras comunes (conocidas como *stop words*) y obtenemos las 1000 palabras más utilizadas en el conjunto de datos. Luego, a estas palabras se les asigna un identificador único, formando un diccionario para one-hot encoding.

Básicamente, el código convierte las oraciones del texto original en secuencias de estos identificadores únicos, traduciendo el lenguaje humano a un formato que un modelo de aprendizaje automático pueda entender.

In [ ]:
def preprocess_string(string):
    # Remove all non-word characters (everything except numbers and letters)
    string = re.sub(r"[^\w\s]", '', string)
    # Replace all runs of whitespaces with no space
    string = re.sub(r"\s+", '', string)
    # replace digits with no space
    string = re.sub(r"\d", '', string)
    return string

In [ ]:
string1 = "💜"
string2 = "Holaaaa!!"
string3 = "o???"

print('Algunos ejemplos:')
print(f"'{string1}' se transforma en: {preprocess_string(string1)}")
print(f"'{string2}' se transforma en: {preprocess_string(string2)}")
print(f"'{string3}' se transforma en: {preprocess_string(string3)}")

In [ ]:
for sentence in X_train:
  print(sentence)
  break

In [ ]:
def tokenize(x_train,y_train,x_val,y_val):

    word_list = []
    stop_words = set(stopwords.words('english'))
    # Recorro cada ejemplo de entrenamiento, en este caso cada review
    for sentence in x_train:
        # Comenzamos el proceso de tokenización
        # pasando todo a minuscula y obteniendo los tokens
        for word in sentence.lower().split():
            word = preprocess_string(word)
            #Filtro por stop words
            if word not in stop_words and word != '':
                word_list.append(word)

    corpus = Counter(word_list)
    #corpus is Counter({'br': 80161, 'movie': 58670, 'film': 52009, ...})

    # sorting on the basis of most common words
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]
    #corpus_ is ['br', 'movie', 'film', 'one', 'like', 'good', 'even',..]


    # creating a dict
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}
    # onehot_dict is {'br': 1, 'movie': 2, 'film': 3, 'one': 4......}


    # tokenize
    final_list_train = []
    final_list_test = []

    # Para cada ejemplo de entrenamiento y validación voy a armar una lista que contenga
    # la ocurrencia de cada palabra dentro de la review
    for sentence in x_train:
            final_list_train.append([onehot_dict[preprocess_string(word)] for word in sentence.lower().split()
                                     if preprocess_string(word) in onehot_dict.keys()])
    for sentence in x_val:
            final_list_test.append([onehot_dict[preprocess_string(word)] for word in sentence.lower().split()
                                    if preprocess_string(word) in onehot_dict.keys()])

    encoded_train = [1 if label =='positive' else 0 for label in y_train]
    encoded_test = [1 if label =='positive' else 0 for label in y_val]
    return np.array(final_list_train), np.array(encoded_train), np.array(final_list_test), np.array(encoded_test), onehot_dict


x_train,y_train,x_val,y_val,vocab = tokenize(X_train,y_train,X_val,y_val)


In [ ]:
print(x_train[10])
print(y_train[10])


In [ ]:
print(x_val[12])
print(y_val[12])

In [ ]:
rev_len_train = [len(i) for i in x_train]
rev_len_val = [len(i) for i in x_val]

pd.Series(rev_len_train).hist(label ='train')
pd.Series(rev_len_val).hist(label='validation')
plt.legend()
plt.title('Distribución de longitud de reviews')
plt.show()

## Preparando los datos para el modelo

Dada la longitud variable de los tokens de cada review, es necesario estandarizarlos para mantener la coherencia. Como la mayoría de las reseñas contienen menos de 500 tokens, estableceremos 500 como longitud fija para todas las reviews.

In [ ]:
x_train.shape

In [ ]:
np.zeros((10, 3),dtype=int)

In [ ]:
def padding(sentences, seq_len):

    features = np.zeros((len(sentences), seq_len),dtype=int)
    for index, review in enumerate(sentences):
        if len(review) != 0:
            features[index, -len(review):] = np.array(review)[:seq_len]
    return features


x_train_pad = padding(x_train,500)
x_val_pad = padding(x_val,500)

In [ ]:
print(x_train_pad.shape)
print(x_val_pad.shape)

In [ ]:
index= 154
print(x_train[index])
print('----')
print(x_train_pad[index][:])

De esta manera, el resultado del padding es transformar cada review (sentence) en un vector de dimensión 500.

## Definimos los Tensores de entrada

Recordemos que la clase `Dataloader` es una clase que permite cargar los datos y recibe los siguientes inputs:

- Un dataset a cargar: Puede ser un tensor, una lista de tensores o cualquier estructura de datos que contenga los datos a modelar.
- `shuffle`: Cuando este parámetro es `True` controla si se deben barajar los datos antes de dividirlos en lotes. Barajar los datos es útil para evitar que el modelo aprenda el orden en el que se presentaron los ejemplos de entrenamiento y, por lo tanto, para mejorar la generalización.
- `batch_size`: Establece el tamaño de lote, es decir, cuántos ejemplos se cargarán a la vez. El conjunto de datos se divide en lotes del tamaño batch_size, y el modelo se entrena en cada lote por separado. Usar lotes en lugar de pasar todos los datos a la vez puede hacer que el entrenamiento sea más eficiente y gestionable.





In [ ]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(x_val_pad), torch.from_numpy(y_val))


# dataloaders
batch_size = 50


# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size, drop_last=True)


# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Training set:')
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample output: \n', sample_y)


# obtain one batch of training data
dataiter = iter(valid_loader)
sample_x, sample_y = next(dataiter)

print('\n')
print('Validation dataset:')
print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample output: \n', sample_y)

## Definimos el modelo



### Usando una capa de embeddings

Documentación:

- [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html): Se utiliza para crear una capa de embedding en una red neuronal. Esta capa se utiliza para aprender representaciones vectoriales densas de palabras en un espacio de características continuas. La llamada `nn.Embedding(vocab_size, embedding_dim)` realiza lo siguiente:

- `vocab_size`: Es el tamaño del vocabulario, es decir, el número total de palabras o ítems únicos en tu conjunto de datos. Cada palabra se mapea a un número entero único, y `vocab_size` representa la cantidad de valores enteros distintos que pueden tomar. Cada valor entero se asocia con un vector de embedding único.

- `embedding_dim`: Especifica la dimensión de los vectores de embedding que se aprenderán para representar cada palabra. Cada palabra se representará como un vector de `embedding_dim` dimensiones.

Cuando se crea una instancia de `nn.Embedding(vocab_size, embedding_dim)`, se está configurando una capa de embedding que entrenará junto con el resto de tu modelo durante el proceso de entrenamiento. Esta capa mapea cada palabra única (identificada por un valor entero) a un vector de embedding correspondiente de `embedding_dim` dimensiones. Estos vectores de embedding se pueden aprender y ajustar para capturar relaciones semánticas entre las palabras en función de los patrones presentes en los datos de entrenamiento.

Cuando se pasa un valor entero (por ejemplo, el índice de una palabra) a esta capa de embedding, devuelve el vector de embedding asociado con ese valor entero. Los vectores de embedding resultantes se utilizan como entrada para la red neuronal, lo que permite a la red aprender representaciones más ricas y significativas de las palabras en lugar de simplemente tratar las palabras como índices enteros. Esto es útil en tareas de procesamiento de lenguaje natural (NLP) y muchas otras aplicaciones de aprendizaje automático.

### LSTM model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self,
                 layers_number, #
                 vocab_size, # Tambien se lo suele escribir como input_size
                 hidden_dim, #
                 embedding_dim, #
                 dropout=0.5):

        super(LSTMModel,self).__init__()

        self.output_dim = output_dim # La cantidad de clases a predecir
        self.hidden_dim = hidden_dim #El numero de nodos en cada capa LSTM (similar a una full conected layer)

        self.layers_number = layers_number # Número de capas LSTM en la red
        self.vocab_size = vocab_size # Tamaño del vocabulario, es decir el número de palabras distintas

        # embedding and LSTM layers
        # Dimension de los vectores de embeddings para representar las palabras
        # Es una matriz que tiene tantas filas como palabras y tantas columnas como la dimension
        # del vector que definimos
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # La capa lstm toma como entrada los embeddings
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=self.hidden_dim, #Representa la cantidad de features en el estado oculto
                           num_layers=layers_number, #Numero de capas recurrentes
                            batch_first=True) #El batch es la primera dimension, nuestros datos tienen la dimension (batch_size, seq_len, features)

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # linear and sigmoid layer
        self.lineal = nn.Linear(self.hidden_dim, output_dim) #Transforma las LSTM en un vector de dimension igual al numero de clases
        self.sigmoid = nn.Sigmoid()

    def forward(self,x,hidden):
        batch_size = x.size(0) # Calcula el tamaño del lote dentro de los datos de entrada

        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: BATCH x SENQUENCE_LENGHT x FEATURES   since batch = True (Features es la cantidad total de palabras)
        #print(embeds.shape)  #[X, Y,Z]
        lstm_out, hidden = self.lstm(embeds, hidden) # Obtenemos las celdas de salidas y un nuevo estado oculto

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) # Redimensiona para tener la forma (batch_size*seq_len, hidden_dim) -> tensor de 2 dimensiones

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.lineal(out)

        # sigmoid function
        sig_out = self.sigmoid(out)

        #Redimensionamos para tener dimension (batch_size, -1),
        #es decir para obtener las predicciones finales de la red LSTM de cada lote
        sig_out = sig_out.view(batch_size, -1)


        sig_out = sig_out[:, -1] # Selecciona la ultima columna de la variable sig_out que contiene, en general, los resultados de la capa lineal

        # return last sigmoid output and hidden state
        return sig_out, hidden

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.layers_number,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.layers_number,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

In [ ]:
layers_number = 2
vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 64
#Como tenemos un problema de clasificacion binaria y aplicamos la funcion sigmoide
# La salida es un tensor unimensional cuyo valor de activacion nos indica la probabilidad
# de pertenencia a la clase
output_dim = 1
hidden_dim = 256


model = LSTMModel(layers_number,
                  vocab_size,
                  hidden_dim,
                  embedding_dim,
                  dropout=0.5)

use_cuda = torch.cuda.is_available()
device = torch.device('cuda') if use_cuda else torch.device('cpu')

#moving to gpu
model.to(device)
print(model)

In [ ]:
# loss and optimization functions

lr=0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


# function to predict accuracy
def acc(pred,label):
    #Nos aseguramos que tanto pred como label tengan las mismas dimensiones
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item() #Cantidad de predicciones correctas


In [ ]:
clip = 5
EPOCHS = 50
valid_loss_min = np.Inf

# train for some number of epochs
epoch_tr_loss = []
epoch_vl_loss = []

epoch_tr_acc = []
epoch_vl_acc = []


#for epoch in trange(EPOCHS):
for epoch in trange(EPOCHS):
  train_losses = []
  train_acc = 0.0
  model.train()

  # initialize hidden state
  # Establecemos el estado inicial en cero
  hidden = model.init_hidden(batch_size)

  # Dado que el dataset tiene x datos y pusimos un batch de y
  # vamos a hacer n pasos de entrenamiento
  for inputs, labels in tqdm(train_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)

      # Creamos nuevas variables para el estado oculto,
      # de otra manera propagariamos a lo largo de toda la historia de entrenamiento teniendo problemas del explosion del gradiente.
      # El estado oculto se actualiza en cada paso de tiempo
      # Con la linea siguiente "devinculamos" el estado oculto de cualquier calculo anterior
      # y solo extraemos sus valores copiandolos con each.data
      hidden = tuple([each.data for each in hidden])

      model.zero_grad()
      output,hidden = model(inputs,hidden) #Realizamos un paso hacia adelante

      # Calculamos la loss y hacemos el backpropagation
      loss = criterion(output.squeeze(), labels.float())
      loss.backward()

      train_losses.append(loss.item())

      # calculating accuracy
      accuracy = acc(output,labels)
      train_acc += accuracy

      #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
      # Limita la norma de los gradientes al valor especificado en clip
      # model.parameters() es una lista de los parámetros del modelo,
      # que son tensores en los que se almacenan los gradientes después de realizar la retropropagación del error.
      nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()

  val_h = model.init_hidden(batch_size)
  val_losses = []
  val_acc = 0.0
  model.eval()
  for inputs, labels in valid_loader:
          val_h = tuple([each.data for each in val_h])

          inputs =  inputs.to(device)
          labels = labels.to(device)


          output, val_h = model(inputs, val_h)
          val_loss = criterion(output.squeeze(), labels.float())


          val_losses.append(val_loss.item())

          accuracy = acc(output,labels)
          val_acc += accuracy

  epoch_train_loss = np.mean(train_losses)
  epoch_val_loss = np.mean(val_losses)

  epoch_train_acc = train_acc/len(train_loader.dataset)
  epoch_val_acc = val_acc/len(valid_loader.dataset)

  epoch_tr_loss.append(epoch_train_loss)
  epoch_vl_loss.append(epoch_val_loss)

  epoch_tr_acc.append(epoch_train_acc)
  epoch_vl_acc.append(epoch_val_acc)

  print(f'Epoch {epoch+1}')
  print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
  print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')

  if epoch_val_loss <= valid_loss_min:
      torch.save(model.state_dict(), 'state_dict.pt')
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
      valid_loss_min = epoch_val_loss
  print(25*'==')

In [ ]:
fig = plt.figure(figsize = (20, 6))
plt.subplot(1, 2, 1)
plt.plot(epoch_tr_acc, label='Train Acc')
plt.plot(epoch_vl_acc, label='Validation Acc')
plt.title("Accuracy")
plt.legend()
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(epoch_tr_loss, label='Train loss')
plt.plot(epoch_vl_loss, label='Validation loss')
plt.title("Loss")
plt.legend()
plt.grid()


plt.show()


In [ ]:
def predict_text(text):
        word_seq = np.array([vocab[preprocess_string(word)] for word in text.split()
                         if preprocess_string(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)
        pad =  torch.from_numpy(padding(word_seq,500))
        inputs = pad.to(device)
        batch_size = 1
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, h)
        return(output.item())


index = 50
print(df['review'][index])
print('='*70)
print(f'Actual sentiment is  : {df["sentiment"][index]}')
print('='*70)
pro = predict_text(df['review'][index])
status = "positive" if pro > 0.5 else "negative"
pro = (1 - pro) if status == "negative" else pro
print(f'Predicted sentiment is {status} with a probability of {pro}')


## Ejercicios extras


Una vez que hemos implementado la arquitectura básica de la red, podemos comenzar a experimentar con distintas modificaciones para lograr mejores resultados. Algunas tareas posibles son:

- Agregar más capas recurrentes
- Probar otros largos de secuencias máximas
- Probar diferentes valores de learning rate

El material de está notebook está fuertemente basado en el material que se encuentra en [datacamp](https://www.datacamp.com/tutorial/nlp-with-pytorch-a-comprehensive-guide).


Otros links útiles:

- [A Beginner’s Guide on Recurrent Neural Networks with PyTorch](https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/)

- [The Ultimate Showdown: RNN vs LSTM vs GRU – Which is the Best?](https://www.shiksha.com/online-courses/articles/rnn-vs-gru-vs-lstm/#:~:text=In%20conclusion%2C%20the%20key%20difference,struggle%20with%20long%2Dterm%20dependencies)

- [RNN hiperparametros y sus valores por default](https://gist.github.com/DavidIsrawi/6c45744c12a4f8fc08bd5b8f7f9e06d8)


- [NLP-Bginner-RNN-GRU-LSTM-Embeddings-GloVe-using-PyTorch](https://github.com/deshwalmahesh/NLP-Bginner-RNN-GRU-LSTM-Embeddings-GloVe-using-PyTorch/blob/master/nlp-beginner-1-rnn-lstm-gru-embeddings-glove.ipynb)

- [Using Imbalanced-Learn to Handle Imbalanced Text Data in NLP](https://www.section.io/engineering-education/using-imbalanced-learn-to-handle-imbalanced-text-data/)